In [1]:
import sqlite3
import pandas as pd

db_path = '../Tutorial 1/waste_for_students.db'

conn = sqlite3.connect(db_path)

df = pd.read_sql("SELECT * FROM locations", conn)
df.set_index('name', inplace=True)
df

,latitude,longitude
name,,
Depot,53.197625,6.612788
1911-945,53.201780,6.560270
2488-933,53.223680,6.535580
1543-519,53.218840,6.555350
1116-116,53.234550,6.561640
...,...,...
1499-978,53.226690,6.581520
0001-1,53.197850,6.611830
1159-1369,53.208940,6.570050


The depot is node (index) 0 as seen above

In [2]:
from pyproj import Geod, CRS

geod = CRS("EPSG:32632").get_geod()

def distance_to_depot(container_name):
    lons = df.loc[['Depot', container_name], 'longitude'].to_list()
    lats = df.loc[['Depot', container_name], 'latitude'].to_list()
    return geod.line_length(lons, lats)

distance_to_depot('2065-881')

5739.79193100201

In [3]:
import requests

url = "http://router.project-osrm.org"

def get_osrm_distance(start_name, end_name, osrm_url="http://router.project-osrm.org"):
    lat1, lon1 = df.loc[start_name]
    lat2, lon2 = df.loc[end_name]
    url = f"{osrm_url}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    params = {
        "overview": "false",
        "annotations": "distance"
    }
    response = requests.get(url, params=params)
    data = response.json()
    if response.status_code == 200 and data["routes"]:
        return data["routes"][0]["distance"]
    else:
        raise HTTPError(f"Got {response.status_code}")

In [4]:
get_osrm_distance('Depot', '2065-881')


9303

Chat GPT says it takes one way streets into account